In [0]:
from azure.storage.blob import BlobServiceClient
import io
import pandas as pd

# Connection string to your Azure Blob Storage account
connection_string = "DefaultEndpointsProtocol=https;AccountName=ijazblob1;AccountKey=OHedg6WdUSruycWuXnyjZceDUXpNk3Mm8fBEFzQFq7mOzGyeH3JVSQ2tKCFwunaEaXhQkdT+7jYT+AStrnKF6Q==;EndpointSuffix=core.windows.net"

# Define the container name and the blob (file) name
container_name = "ijaz-container"
blob_name = "data.csv"

# Initialize the BlobServiceClient using the connection string
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

# Get a BlobClient for the blob you want to read
blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)

# Download the blob's content into memory (as a stream)
blob_data = blob_client.download_blob()

# Read the content of the blob as a string (this will be CSV data)
csv_data = blob_data.readall()

# Convert the CSV data into a pandas DataFrame for easy manipulation
data_frame = pd.read_csv(io.BytesIO(csv_data))

# Print the DataFrame to see the contents
print(data_frame)

   product_id     product_name  ... customer_location  target_column
0        2001           Tablet  ...        New Jersey              1
1        2002       Smartwatch  ...            Nevada              1
2        2003        Microwave  ...          Illinois              0
3        2004         Speakers  ...           Florida              1
4        2005             Desk  ...           Arizona              0
5        2006  Air Conditioner  ...             Texas              1
6        2007        Projector  ...            Oregon              1
7        2008       Dishwasher  ...              Ohio              0
8        2009          Toaster  ...        California              1
9        2010       VR Headset  ...          Michigan              1

[10 rows x 9 columns]


In [0]:
import pandas as pd
import numpy as np

# Assuming data_frame is your cleaned DataFrame

# 1. Cumulative Sum for 'quantity_sold'
data_frame['cumulative_quantity_sold'] = data_frame['quantity_sold'].cumsum()

# 2. Rolling Averages
# We can calculate rolling averages for numerical columns like 'price', 'quantity_sold', and 'discount'
# For simplicity, we use a 3-row rolling window for each of these columns

data_frame['rolling_avg_price'] = data_frame['price'].rolling(window=3).mean()
data_frame['rolling_avg_quantity_sold'] = data_frame['quantity_sold'].rolling(window=3).mean()
data_frame['rolling_avg_discount'] = data_frame['discount'].rolling(window=3).mean()

# 3. Date-based Features (if you have a 'date' column)
# For this example, assume the date column exists and is named 'date'
# You can extract year, month, day, weekday, etc.
# If there's no date column, you can skip this step

# Example: Assuming 'date' is a datetime column
if 'date' in data_frame.columns:
    data_frame['year'] = data_frame['date'].dt.year
    data_frame['month'] = data_frame['date'].dt.month
    data_frame['day'] = data_frame['date'].dt.day
    data_frame['weekday'] = data_frame['date'].dt.weekday

# 4. One-Hot Encoding for 'category' and 'customer_location'
# One-Hot Encoding using pandas' get_dummies
if 'category' in data_frame.columns and 'customer_location' in data_frame.columns:
    data_frame = pd.get_dummies(data_frame, columns=['category', 'customer_location'], drop_first=True)

# 5. Creating Interaction Features
# For example, the interaction between 'price' and 'quantity_sold' might be useful
data_frame['price_quantity_interaction'] = data_frame['price'] * data_frame['quantity_sold']

# 6. Feature Transformation: Log Transformation (for highly skewed features)
# Apply log transformation to 'price' if it is highly skewed
if data_frame['price'].skew() > 1:  # check if skew is significant
    data_frame['log_price'] = np.log1p(data_frame['price'])  # log1p handles 0 values safely

# Apply log transformation to 'quantity_sold' if needed
if data_frame['quantity_sold'].skew() > 1:
    data_frame['log_quantity_sold'] = np.log1p(data_frame['quantity_sold'])

# 7. Additional Custom Feature (e.g., price per unit sold)
data_frame['price_per_unit'] = data_frame['price'] / (data_frame['quantity_sold'] + 1)

# 8. Remove any unnecessary columns (optional)
# Example: If 'product_name' and 'target_column' aren't useful for modeling, drop them
columns_to_drop = ['product_name', 'target_column']
existing_columns_to_drop = [col for col in columns_to_drop if col in data_frame.columns]
data_frame = data_frame.drop(columns=existing_columns_to_drop)

# Show the final engineered DataFrame
print(data_frame)

   product_id  price  ...  price_quantity_interaction  price_per_unit
0        2001    500  ...                       10000       23.809524
1        2002    250  ...                        8750        6.944444
2        2003    150  ...                        2250        9.375000
3        2004    120  ...                        2160        6.315789
4        2005    180  ...                        2160       13.846154
5        2006    700  ...                        6300       70.000000
6        2007    300  ...                        8400       10.344828
7        2008    800  ...                        8000       72.727273
8        2009     60  ...                        1500        2.307692
9        2010    350  ...                        5600       20.588235

[10 rows x 23 columns]


In [0]:

from azure.storage.blob import BlobServiceClient
import io
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Connection string to your Azure Blob Storage account
connection_string = "DefaultEndpointsProtocol=https;AccountName=ijazblob1;AccountKey=OHedg6WdUSruycWuXnyjZceDUXpNk3Mm8fBEFzQFq7mOzGyeH3JVSQ2tKCFwunaEaXhQkdT+7jYT+AStrnKF6Q==;EndpointSuffix=core.windows.net"

# Define the container name and the blob (file) name
container_name = "ijaz-container"
blob_name = "data.csv"

# Initialize the BlobServiceClient using the connection string
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

# Get a BlobClient for the blob you want to read
blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)

# Download the blob's content into memory (as a stream)
blob_data = blob_client.download_blob()

# Read the content of the blob as a string (this will be CSV data)
csv_data = blob_data.readall()

# Convert the CSV data into a pandas DataFrame for easy manipulation
data_frame = pd.read_csv(io.BytesIO(csv_data))

# Print the DataFrame to see the contents
print(data_frame)

# Assuming data_frame is your cleaned DataFrame

# 1. Cumulative Sum for 'quantity_sold'
data_frame['cumulative_quantity_sold'] = data_frame['quantity_sold'].cumsum()

# 2. Rolling Averages
data_frame['rolling_avg_price'] = data_frame['price'].rolling(window=3).mean()
data_frame['rolling_avg_quantity_sold'] = data_frame['quantity_sold'].rolling(window=3).mean()
data_frame['rolling_avg_discount'] = data_frame['discount'].rolling(window=3).mean()

# 3. One-Hot Encoding for 'category' and 'customer_location'
data_frame = pd.get_dummies(data_frame, columns=['category', 'customer_location'], drop_first=True)

# 4. Creating Interaction Features
data_frame['price_quantity_interaction'] = data_frame['price'] * data_frame['quantity_sold']

# 5. Feature Transformation: Log Transformation (for highly skewed features)
if data_frame['price'].skew() > 1:
    data_frame['log_price'] = np.log1p(data_frame['price'])

if data_frame['quantity_sold'].skew() > 1:
    data_frame['log_quantity_sold'] = np.log1p(data_frame['quantity_sold'])

# 6. Additional Custom Feature (e.g., price per unit sold)
data_frame['price_per_unit'] = data_frame['price'] / (data_frame['quantity_sold'] + 1)

# 7. Remove any unnecessary columns if they exist
columns_to_drop = ['product_name', 'target_column']
data_frame = data_frame.drop(columns=[col for col in columns_to_drop if col in data_frame.columns])

# Define X (features) and y (target variable) if 'target_column' exists
if 'target_column' in data_frame.columns:
    X = data_frame.drop(columns=['target_column'])  # Features
    y = data_frame['target_column']  # Target variable

    # Drop rows with NaN values in the feature matrix
    X = X.dropna()
    y = y[X.index]  # Ensure that y matches the rows in X

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Show the final engineered DataFrame
    print(X_train)
else:
    print("The 'target_column' does not exist in the DataFrame.")

   product_id     product_name  ... customer_location  target_column
0        2001           Tablet  ...        New Jersey              1
1        2002       Smartwatch  ...            Nevada              1
2        2003        Microwave  ...          Illinois              0
3        2004         Speakers  ...           Florida              1
4        2005             Desk  ...           Arizona              0
5        2006  Air Conditioner  ...             Texas              1
6        2007        Projector  ...            Oregon              1
7        2008       Dishwasher  ...              Ohio              0
8        2009          Toaster  ...        California              1
9        2010       VR Headset  ...          Michigan              1

[10 rows x 9 columns]
The 'target_column' does not exist in the DataFrame.


In [0]:
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Define X and y
X = np.array([[1, 2], [np.nan, 3], [7, 6]])
y = np.array([1, 2, 3])

# Impute missing values in X
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

# Split the data into training and testing sets with a smaller test size
X_train, X_test, y_train, y_test = train_test_split(
    X_imputed, y, test_size=0.33, random_state=42
)

# Train the Linear Regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error (MAE): {mae}")

mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error (MSE): {mse}")

rmse = np.sqrt(mse)
print(f"Root Mean Squared Error (RMSE): {rmse}")

r2 = r2_score(y_test, y_pred)
print(f"R-squared (R²): {r2}")

Uploading artifacts:   0%|          | 0/9 [00:00<?, ?it/s]

Mean Absolute Error (MAE): 0.3333333333333339
Mean Squared Error (MSE): 0.11111111111111151
Root Mean Squared Error (RMSE): 0.3333333333333339
R-squared (R²): nan


/databricks/python/lib/python3.12/site-packages/sklearn/metrics/_regression.py:1187: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


In [0]:
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# Assuming 'X' is the feature matrix and 'y' is the target variable
# Impute missing values in X
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_imputed, y, test_size=0.2, random_state=42)

# Train the Linear Regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model using RMSE and R² on the test set
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error (MAE): {mae}")

mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error (MSE): {mse}")

rmse = np.sqrt(mse)
print(f"Root Mean Squared Error (RMSE): {rmse}")

r2 = r2_score(y_test, y_pred)
print(f"R-squared (R²): {r2}")

# Cross-validation evaluation using RMSE and R²
kf = KFold(n_splits=min(5, len(X_imputed)), shuffle=True, random_state=42)

# Using built-in scoring methods for cross-validation
cv_rmse_scores = cross_val_score(model, X_imputed, y, cv=kf, scoring='neg_root_mean_squared_error')
cv_r2_scores = cross_val_score(model, X_imputed, y, cv=kf, scoring='r2')

# Convert negative RMSE to positive values
cv_rmse_scores = -cv_rmse_scores

print(f"Cross-Validation RMSE Scores: {cv_rmse_scores}")
print(f"Mean CV RMSE: {cv_rmse_scores.mean()}")

print(f"Cross-Validation R² Scores: {cv_r2_scores}")
print(f"Mean CV R²: {cv_r2_scores.mean()}")

Uploading artifacts:   0%|          | 0/9 [00:00<?, ?it/s]

Mean Absolute Error (MAE): 0.3333333333333339
Mean Squared Error (MSE): 0.11111111111111151
Root Mean Squared Error (RMSE): 0.3333333333333339


/databricks/python/lib/python3.12/site-packages/sklearn/metrics/_regression.py:1187: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


R-squared (R²): nan
Cross-Validation RMSE Scores: [0.33333333 0.15384615 0.2       ]
Mean CV RMSE: 0.229059829059829
Cross-Validation R² Scores: [nan nan nan]
Mean CV R²: nan


/databricks/python/lib/python3.12/site-packages/sklearn/metrics/_regression.py:1187: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/databricks/python/lib/python3.12/site-packages/sklearn/metrics/_regression.py:1187: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/databricks/python/lib/python3.12/site-packages/sklearn/metrics/_regression.py:1187: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


In [0]:
import joblib
from azure.storage.blob import BlobServiceClient
import os
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Assuming 'X' is the feature matrix and 'y' is the target variable
# Example: Train a linear regression model (replace with your model and training logic)
# Here, I use random data as an example. Replace this with your actual dataset.
X = np.random.rand(100, 5)  # 100 samples, 5 features
y = np.random.rand(100)  # 100 target values

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Linear Regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error (MSE): {mse}")

# Save the trained model to a local file
model_filename = "linear_regression_model.pkl"
joblib.dump(model, model_filename)

# Azure Blob Storage details
connection_string = "DefaultEndpointsProtocol=https;AccountName=ijazblob1;AccountKey=OHedg6WdUSruycWuXnyjZceDUXpNk3Mm8fBEFzQFq7mOzGyeH3JVSQ2tKCFwunaEaXhQkdT+7jYT+AStrnKF6Q==;EndpointSuffix=core.windows.net"
container_name = "ijaz-container"
model_blob_name = "linear_regression_model.pkl"  # Blob name in the container

# Initialize the BlobServiceClient using the connection string
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

# Get the BlobClient for the model file
blob_client = blob_service_client.get_blob_client(container=container_name, blob=model_blob_name)

# Upload the model file to Azure Blob Storage
try:
    # Open the model file in binary mode and upload to Blob Storage
    with open(model_filename, "rb") as data:
        blob_client.upload_blob(data, overwrite=True)  # overwrite=True to replace any existing file

    print(f"Model successfully uploaded to Azure Blob Storage as {model_blob_name}")

    # Optionally, remove the local model file after uploading
    os.remove(model_filename)

except Exception as e:
    print(f"Error uploading model to Azure Blob Storage: {e}")

Uploading artifacts:   0%|          | 0/9 [00:00<?, ?it/s]

Mean Squared Error (MSE): 0.10014911552688335
Model successfully uploaded to Azure Blob Storage as linear_regression_model.pkl
